# Lab 1: CIFAR-10 图像分类任务 - 从头实现数据预处理和加载

## 作业描述

在上一次作业中，我们用 MLP 对 CIFAR-10 数据集进行了分类，但是数据处理部分是使用了 PyTorch 和 torchvision 库中的内置函数，为了深入了解图像预处理和数据加载的细节，我们将手动实现这些函数，并改进为更高效的批处理方式。在后续的对比中，我们将会发现手动实现的函数在速度上显著优于内置函数。

## 任务
补全 [`data_scratch.py`](./data_scratch.py) 中的以下数据预处理代码：
- `Normalize`：根据指定的均值和标准差对图像进行归一化。(15pts)
- `ToTensor`：将图像像素值缩放到 [0.0, 1.0]，并调整维度顺序为 (B, C, H, W)。(10pts)
- `RandomHorizontalFlip`：以指定的概率对图像进行随机水平翻转。(20pts)
- `Compose`：组合多个图像变换操作。(5pts)

## Setup Code

首先，我们通过 `%load_ext autoreload` 魔术命令设置自动重新加载，这样在修改了 .py 文件后，更改可以立即在notebook中生效。

In [1]:
%load_ext autoreload
%autoreload 2

`show_code` 是我们在[`utils.py`](./utils.py)中定义的一个辅助函数，用于在Jupyter Notebook中显示函数的源代码。

In [2]:
from utils import show_code

show_code(show_code)

def show_code(func):
    """用于在Jupyter Notebook中显示函数的源代码"""
    source_code = inspect.getsource(func)
    print(source_code)



## 数据处理 pipline 的简洁实现

在 [`data_concise.py`](./data_concise.py) 中，我们使用了 PyTorch 和 torchvision 库中的内置函数，这为我们提供了一个简单的参考实现。我们可以查看 `load_data_concise` 函数的源代码如下：

In [3]:
from data_concise import load_data_concise

show_code(load_data_concise)

def load_data_concise(root="cifar10", batch_size=64):
    MEAN = (0.4914, 0.4822, 0.4465)
    STD = (0.2023, 0.1994, 0.2010)
    train_trans = T.Compose([T.ToTensor(), T.RandomHorizontalFlip(0.5), T.Normalize(MEAN, STD)])
    test_trans = T.Compose([T.ToTensor(), T.Normalize(MEAN, STD)])

    train_data = torchvision.datasets.CIFAR10(root=root, train=True, download=True, transform=train_trans)
    test_data = torchvision.datasets.CIFAR10(root=root, train=False, download=True, transform=test_trans)

    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, num_workers=4)
    return train_loader, test_loader



这里的 `MEAN = (0.4914, 0.4822, 0.4465)` 和 `STD = (0.2023, 0.1994, 0.2010)` 是在整个训练集上计算得到的，其计算流程如下：
1. 对训练集中所有图像的像素值求均值和标准差，在RGB通道上分别计算。
2. 对所有图像的均值和标准差求平均值，得到最终的均值和标准差。

In [4]:
import torch
import torchvision
import torchvision.transforms as T

train_data = torchvision.datasets.CIFAR10(root="cifar10", train=True, download=True, transform=T.ToTensor())
# 将所有的图片数据放在一个张量中，然后计算均值和标准差
train_data_tensor = torch.stack([img for img, _ in train_data], dim=0)
cifar10_mean = train_data_tensor.mean(dim=[2, 3]).mean(dim=0)
cifar10_std = train_data_tensor.std(dim=[2, 3]).mean(dim=0)
print(f"mean: {cifar10_mean}, std: {cifar10_std}")

Files already downloaded and verified
mean: tensor([0.4914, 0.4822, 0.4465]), std: tensor([0.2023, 0.1994, 0.2010])


提前计算好整体的 mean 和 std，而不是分别计算每个batch的mean和std，这样可以节省时间。

接下来我们可以查看`test_loader`的输出结果，用于后续验证我们手动实现的结果是否正确。

In [5]:
train_loader, test_loader = load_data_concise(root="cifar10", batch_size=64)
for images, labels in test_loader:
    print(images.shape, labels.shape)
    break

Files already downloaded and verified
Files already downloaded and verified
torch.Size([64, 3, 32, 32]) torch.Size([64])


## 数据处理 pipline 的从头实现

### 数据集读取

在torchvision内置函数中，它将CIFAR-10数据集下载到本地并分成多个batch，在数据量较大时，分成多个batch是非常有必要的，因为这样读取时可以节省内存。
但是在这里，整个`CIFAR-10`数据集只有160MB，因此我们完全可以将整个数据集保存为一个`train.pt`和一个`test.pt`，然后一次性读取。

此外，我们在读取时，通过设置`torch.load`函数中的`map_location`参数，可以将数据加载到GPU上，这样训练时就不需要再将数据从CPU传输到GPU，可以加快训练速度。

In [6]:
import data_scratch

show_code(data_scratch.load_cifar10)

def load_cifar10(root="cifar10", train=True):
    """直接将整个数据集载入到 GPU 上, 可以加快训练速度"""
    save_path = os.path.join(root, "train.pt" if train else "test.pt")
    if not os.path.exists(save_path):
        dset = torchvision.datasets.CIFAR10(root, download=True, train=train)
        images = torch.tensor(dset.data, dtype=torch.float32)
        labels = torch.tensor(dset.targets, dtype=torch.int64)
        torch.save({"images": images, "labels": labels}, save_path)
    data = torch.load(save_path, map_location=device)
    return data["images"], data["labels"]



### 定义数据集class

我们仿照`torchvision.datasets.CIFAR10`的API，定义了一个`CifarDataset`类，注意这次我们没有像HW1中那样从`torch.utils.data.Dataset`继承，这意味着这些功能全是我们自己实现的，数据集的读取和预处理将不再是黑盒，我们可以自由地对数据集进行处理。

In [7]:
show_code(data_scratch.CifarDataset)

class CifarDataset:
    def __init__(self, root, train=True, transform=None):
        self.images, self.labels = load_cifar10(root, train=train)
        self.train = train
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        images = self.images[idx]
        labels = self.labels[idx]
        if self.transform is not None:
            images = self.transform(images)
        return images, labels



### 数据预处理

在`CifarDataset`类中，我们传入了一个`transform`参数，当我们调用`__getitem__`函数时，就会自动对数据进行预处理变换。

现在，我们需要实现这些图像预处理的代码，请编辑[`data_scratch.py`](./data_scratch.py) 文件，补全`Normalize`、`ToTensor`、`RandomHorizontalFlip`以及`Compose`类。

#### ToTensor 类

在torchvision的`ToTensor`类中，它实现了三个功能：
1. 将PIL Image或者numpy.ndarray类型的数据转换为torch.FloatTensor
2. 将数据的范围从[0, 255]缩放到[0.0, 1.0]
3. 将数据的维度从(H, W, C)转换为(C, H, W)

这里由于我们的输入数据是torch.FloatTensor类型，因此我们只需要实现后两个功能即可。

注意，原始的`ToTensor`一次只能处理一张图片，而我们的实现中应该能够同时处理多张图片，即输入的数据维度是(B, H, W, C)，输出的数据维度是(B, C, H, W)。

Hint: 维度转换可以使用 torch.Tensor.permute 函数, https://pytorch.org/docs/stable/generated/torch.permute.html#torch.permute

我们在 [`test_data_scratch.py`](./test_data_scratch.py) 中为你准备了测试代码，你可以通过运行下面的代码来测试你的实现。

In [8]:
import test_data_scratch as tds

tds.test_ToTensor()

ToTensor test passed.


#### RandomHorizontalFlip 类

对输入的每一张图片, 以独立的概率 $p$ 进行水平翻转, 注意使用批量处理的方式而不是逐个处理每张图片.

Hint: 翻转可以使用 torch.flip 函数, https://pytorch.org/docs/stable/generated/torch.flip.html

In [9]:
tds.test_RandomHorizontalFlip()

RandomHorizontalFlip test passed.


####  Normalize 类

实现 `Normalize` 类，它应该能够接受图像的均值和标准差，并使用这些值来归一化图像，你的归一化应该是对于每个通道独立进行的。

注意，输入的均值和标准差是一个长度为3的列表/元组，分别对应RGB三个通道的均值和标准差。

In [10]:
tds.test_Normalize()

Normalize test passed.


#### Compose 类

实现 `Compose` 类，它接受一个列表，列表中的元素就是上面我们定义的`Normalize`、`ToTensor`、`RandomHorizontalFlip`类的实例，然后按照列表中的顺序依次对图像进行处理。

In [11]:
tds.test_Compose()

Compose test passed.


### 数据载入

我们从头实现了一个`CifarLoader`类，它的功能和`torch.utils.data.DataLoader`类似，具有相同的API，虽然我们的实现可能没有内置函数那么完善，例如缺少多线程读取等功能，但是由于我们的数据已经全部存放在GPU中，因此这些功能对我们来说并不重要。

In [12]:
show_code(data_scratch.CifarLoader)

class CifarLoader:
    def __init__(self, dataset: CifarDataset, batch_size=512, shuffle=True):
        self.dataset = dataset
        self.batch_size = batch_size
        self.train = dataset.train
        if self.train and shuffle:
            self.shuffle = True
        else:
            self.shuffle = False
        self.data_num = len(dataset)

    def __len__(self):
        """返回 batch 的个数"""
        return ceil(self.data_num / self.batch_size)

    def __iter__(self):
        """使用迭代器返回 image 和 label"""
        indices = torch.randperm(self.data_num) if self.shuffle else torch.arange(self.data_num)
        for i in range(len(self)):
            idxs = indices[i * self.batch_size : (i + 1) * self.batch_size]
            yield self.dataset[idxs]



接下来，类似`load_data_concise`，我们定义了一个`load_data_scratch`函数，可以比较两者输出的结果是否一致。

In [13]:
train_loader1, test_loader1 = load_data_concise(root="cifar10", batch_size=64)
for i, (images, labels) in enumerate(test_loader1):
    print(images[0])
    break

Files already downloaded and verified
Files already downloaded and verified
tensor([[[ 0.6338,  0.6531,  0.7694,  ...,  0.2267,  0.0134, -0.1804],
         [ 0.5174,  0.4981,  0.6531,  ...,  0.2073, -0.0060, -0.1223],
         [ 0.4981,  0.4981,  0.6338,  ...,  0.2654,  0.0910, -0.1029],
         ...,
         [-1.1109, -1.6149, -1.8281,  ..., -1.6924, -2.1771, -1.6537],
         [-1.2466, -1.4792, -1.7506,  ..., -1.9251, -1.8669, -2.0414],
         [-1.3823, -1.3435, -1.5567,  ..., -1.9638, -1.7700, -2.0220]],

        [[-0.2156, -0.2352, -0.1369,  ..., -0.5499, -0.6286, -0.7466],
         [-0.2156, -0.2549, -0.1762,  ..., -0.5499, -0.6286, -0.6876],
         [-0.2549, -0.2746, -0.2352,  ..., -0.4909, -0.5499, -0.6679],
         ...,
         [ 0.0204, -0.4516, -0.6876,  ..., -0.5106, -1.1596, -0.7466],
         [-0.1369, -0.4122, -0.7466,  ..., -0.8056, -0.8056, -1.1596],
         [-0.3139, -0.3532, -0.6679,  ..., -0.9039, -0.7662, -1.1006]],

        [[-1.2654, -1.3044, -1.2264,  ..

In [14]:
from data_scratch import load_data_scrach

train_loader2, test_loader2 = load_data_scrach(root="cifar10", batch_size=64)
for i, (images, labels) in enumerate(test_loader2):
    print(images[0])
    break

tensor([[[ 0.6338,  0.6531,  0.7694,  ...,  0.2267,  0.0134, -0.1804],
         [ 0.5174,  0.4981,  0.6531,  ...,  0.2073, -0.0060, -0.1223],
         [ 0.4981,  0.4981,  0.6338,  ...,  0.2654,  0.0910, -0.1029],
         ...,
         [-1.1109, -1.6149, -1.8281,  ..., -1.6924, -2.1771, -1.6537],
         [-1.2466, -1.4792, -1.7506,  ..., -1.9251, -1.8669, -2.0414],
         [-1.3823, -1.3435, -1.5567,  ..., -1.9638, -1.7700, -2.0220]],

        [[-0.2156, -0.2352, -0.1369,  ..., -0.5499, -0.6286, -0.7466],
         [-0.2156, -0.2549, -0.1762,  ..., -0.5499, -0.6286, -0.6876],
         [-0.2549, -0.2746, -0.2352,  ..., -0.4909, -0.5499, -0.6679],
         ...,
         [ 0.0204, -0.4516, -0.6876,  ..., -0.5106, -1.1596, -0.7466],
         [-0.1369, -0.4122, -0.7466,  ..., -0.8056, -0.8056, -1.1596],
         [-0.3139, -0.3532, -0.6679,  ..., -0.9039, -0.7662, -1.1006]],

        [[-1.2654, -1.3044, -1.2264,  ..., -1.5190, -1.5190, -1.5776],
         [-1.2264, -1.4410, -1.3434,  ..., -1

## 训练时间对比

在上一次作业中，我们实现了`train_epoch`和`eval_model`函数，并完成了整个训练流程。
现在我们继续使用这两个函数，但我们将完整的训练流程封装到 [`train.py`](./train.py) 中的 `run_train` 函数中，通过修改`hyp`可以调整训练的超参数。

神经网络部分，我这里使用了一个简单的三层卷积神经网络，你可以在 [`model.py`](./model.py) 中查看它的实现。

In [15]:
import torch.nn as nn
import torch.nn.functional as F


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

接下来，我们通过控制 `config['use_scratch']` 来选择使用scratch实现还是concise实现，观察输出的`total_time`，比较两者的训练时间。

In [16]:
from train import run_train

config = {
    "train_epochs": 10,
    "batch_size": 512,
    "lr": 1e-3,
    "use_scratch": True,
}

print("Using scratch data loader")
run_train(config)

Using scratch data loader


----------------------------------------------------------------------------------
|  epoch  |  train_loss  |  test_loss  |  train_acc  |  test_acc  |  total_time  |
----------------------------------------------------------------------------------
|      1  |      1.6387  |     1.3934  |     0.4097  |    0.5044  |      0.9045  |
|      2  |      1.2842  |     1.1816  |     0.5438  |    0.5818  |      1.5080  |
|      3  |      1.1132  |     1.0844  |     0.6088  |    0.6154  |      2.1124  |
|      4  |      1.0087  |     0.9945  |     0.6466  |    0.6545  |      2.7203  |
|      5  |      0.9377  |     0.9404  |     0.6766  |    0.6732  |      3.3265  |
|      6  |      0.8890  |     0.9123  |     0.6920  |    0.6815  |      3.9334  |
|      7  |      0.8442  |     0.9056  |     0.7078  |    0.6834  |      4.5377  |
|      8  |      0.8124  |     0.8443  |     0.7205  |    0.7072  |      5.1428  |
|      9  |      0.7740  |     0.8362  |     0.7331  |    0.7109  |      5.7479  |
|   

In [17]:
print("Using concise data loader")
config["use_scratch"] = False
run_train(config)

Using concise data loader
Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------------------------
|  epoch  |  train_loss  |  test_loss  |  train_acc  |  test_acc  |  total_time  |
----------------------------------------------------------------------------------
|      1  |      1.6165  |     1.3631  |     0.4145  |    0.5161  |      1.1985  |
|      2  |      1.2698  |     1.1926  |     0.5504  |    0.5860  |      2.5234  |
|      3  |      1.1210  |     1.0954  |     0.6073  |    0.6137  |      3.8796  |
|      4  |      1.0343  |     1.0340  |     0.6379  |    0.6387  |      5.0863  |
|      5  |      0.9598  |     0.9468  |     0.6648  |    0.6657  |      6.3069  |
|      6  |      0.9123  |     0.9141  |     0.6848  |    0.6812  |      7.5565  |
|      7  |      0.8594  |     0.8972  |     0.7025  |    0.6893  |      8.8109  |
|      8  |      0.8253  |     0.8910  |     0.7150  |    0.6912  | 

可以看到，手动实现的函数在速度上显著优于内置函数，其主要原因在于：内置函数每次训练时都要把数据从CPU传输到GPU，这个过程是非常耗时的。

而在我们的实现中，数据一直存放在GPU上，因此不需要搬运。但是当网络规模变大时，时间消耗的大头会转移到网络的计算上，此时数据传输的时间损耗就会变得没那么明显。